In [ ]:
import pandas as pd
from util_func import *
from util_func import (
    out_dir,
    survey_processed_dir,
    tour_extract_wkday_dir,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
out_filepath = out_dir / "04a_TNCTrips_NumPax.xlsx"

In [ ]:
## Process trip records
raw_trips = pd.read_csv(survey_processed_dir / "trip.csv")
if "linked_trip_id" in raw_trips.columns:
    tripid_col = "linked_trip_id"
else:
    tripid_col = "trip_num"
raw_trips = raw_trips[
    [
        "hh_id",
        "person_num",
        tripid_col,
        "mode_uber",
        "mode_lyft",
        "num_travelers",
        "tnc_pooled",
        "mode_type_imputed",
    ]
]
raw_trips = raw_trips.rename(
    columns={"hh_id": "hhno", "person_num": "pno", tripid_col: "tsvid"}
)

trip_df = pd.read_csv(tour_extract_wkday_dir / "trip-assign_day.csv")
trip_df = link_dt(trip_df)
trip_df = trip_df.loc[trip_df["mode"] == 9,]
trip_df = trip_df.merge(raw_trips, how="left")
trip_df = trip_df[trip_df["mode_type_imputed"] != 4]  # remove taxi trips

trip_df["dpurp2"] = trip_df["dpurp"]
trip_df.loc[trip_df["dpurp"] == 0, "dpurp2"] = trip_df.loc[
    trip_df["dpurp"] == 0, "opurp"
]

trip_df["tnc_type"] = 3  # prem/other
trip_df.loc[(trip_df["mode_uber"] == 1) | (trip_df["mode_lyft"] == 1), "tnc_type"] = (
    1  # pooled
)
trip_df.loc[(trip_df["mode_uber"] == 2) | (trip_df["mode_lyft"] == 2), "tnc_type"] = (
    2  # regular
)

trip_df["tnc_pooled"][trip_df["tnc_pooled"] > 5] = 1
trip_df["tnc_occ"] = trip_df["num_travelers"]
trip_df.loc[
    (trip_df["tnc_type"] == 1) & (trip_df["tnc_pooled"] > 0), "tnc_occ"
] += trip_df.loc[(trip_df["tnc_type"] == 1) & (trip_df["tnc_pooled"] > 0), "tnc_pooled"]
trip_df["tnc_occ_cat"] = trip_df["tnc_occ"]
trip_df.loc[trip_df["tnc_occ_cat"] > 5, "tnc_occ_cat"] = 5

trip_df["count"] = 1
trip_df = trip_df[(trip_df["trexpfac"] > 0) & (trip_df["mode"] > 0)]
trip_df = trip_df[(trip_df["otaz"] > 0) & (trip_df["dtaz"] > 0)]

In [ ]:
col_dict = {
    "dpurp": {
        "desc": "DPurp",
        "col": "dpurp",
        "vals": range(0, 8),
        "labels": [
            "1_Home",
            "2_Work",
            "3_School",
            "4_Escort",
            "5_PersBus",
            "6_Shop",
            "7_Meal",
            "8_SocRec",
        ],
    },
    "dpurp2": {
        "desc": "Non-Home Purp",
        "col": "dpurp2",
        "vals": range(1, 8),
        "labels": [
            "1_Work",
            "2_School",
            "3_Escort",
            "4_PersBus",
            "5_Shop",
            "6_Meal",
            "7_SocRec",
        ],
    },
    "tnc_occ_cat": {
        "desc": "NumPax",
        "col": "tnc_occ_cat",
        "vals": range(1, 6),
        "labels": ["1", "2", "3", "4", "5+"],
    },
    "tncmode": {
        "desc": "TNCMode",
        "col": "tnc_type",
        "vals": range(1, 4),
        "labels": ["2_Pool", "1_Reg", "3_PremOth"],
    },
}

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
row = 0
sname = "Weekday"
d1_dict = col_dict["tnc_occ_cat"]

title = "TNC Trips by " + d1_dict["desc"]
tab = prep_data_1d(
    trip_df,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)
row = write_to_excel(tab, sname, title, row)

title = "Column Shares by " + d1_dict["desc"]
row = write_to_excel(getSharesIdx(tab.copy()), sname, title, row)

tab2 = tab.copy()
tab2.iloc[-1, -1] = tab2.iloc[-1, 0]

title = "Column Shares 95% CI by " + d1_dict["desc"]
row = write_to_excel(getSharesIdxCI95(tab.copy()), sname, title, row)

tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
wt_cols = ["count", "trexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

for key in ["dpurp2", "tncmode"]:
    d2_dict = col_dict[key]
    row = 0
    sname = d2_dict["desc"]

    for wc, wd in zip(wt_cols, wt_desc):
        title = "TNC Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        tab, tab_fmt = prep_data_2d(
            trip_df,
            d1_dict["col"],
            d1_dict["vals"],
            d1_dict["labels"],
            d2_dict["col"],
            d2_dict["vals"],
            d2_dict["labels"],
            wc,
        )
        row = write_to_excel(tab.astype("float64"), sname, title, row)

        if wc == "count":
            tab2 = tab.copy()
        else:
            tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

        title = (
            "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        )
        row = write_to_excel(getSharesIdx(tab.copy()), sname, title, row)

        title = (
            "Column Shares 95% CI by "
            + d2_dict["desc"]
            + " and "
            + d1_dict["desc"]
            + " "
            + wd
        )
        row = write_to_excel(getSharesIdxCI95(tab2.copy()), sname, title, row)
    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()